# Imports

In [1]:
from functools import reduce
from pathlib import Path
import datetime

import pandas as pd
from wmfdata import hive
from wmfdata.utils import print_err, pd_display_all

# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/diversity_metrics.tsv"

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")

# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_next_month_first_day": str((metrics_month+1).asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics

In [4]:
queries = {
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },

    "global_south_new_editor_retention": {
        "file": "queries/global_south_new_editor_retention.hql",
        "engine": "hive"
    },
    
    "global_south_net_new_content": {
        "file": "queries/global_south_net_new_content.hql",
        "engine": "hive"
    },
    
    "global_south_wikidata_entities": {
        "file": "queries/global_south_net_new_wikidata.hql",
        "engine": "hive"
    },
    
    "global_north_edits_editors": {
        "file": "queries/global_north_edits_editors.hql",
        "engine": "hive"
    },

    "global_north_new_editor_retention": {
        "file": "queries/global_north_new_editor_retention.hql",
        "engine": "hive"
    },
    
    "global_north_net_new_content": {
        "file": "queries/global_north_net_new_content.hql",
        "engine": "hive"
    },
    "global_north_wikidata_entities": {
        "file": "queries/global_north_net_new_wikidata.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running global_south_edits_editors on hive...


/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
Running global_south_new_editor_retention on hive...
/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Running global_south_net_new_content on hive...
/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Running global_south_wikidata_entities on hive...
/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Running global_north_edits_editors on hive...
/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Running global_north_new_editor_retention on hive...
/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Running global_north_net_new_content on hive...


/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
Running global_north_wikidata_entities on hive...


/home/iflorez/.conda/envs/2023-03-31T21.45.57_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Combining and saving metrics

In [5]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

In [6]:
# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()
new_metrics = new_metrics.iloc[:,:].astype(float)

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,global_north_active_editors,global_north_edits,global_north_net_new_content,global_north_new_editor_retention,global_north_nonbot_edits,global_north_wikidata_entities,global_south_active_editors,global_south_edits,global_south_net_new_content,global_south_new_editor_retention,global_south_nonbot_edits,global_south_wikidata_entities
month,,,,,,,,,,,,
2022-11-01,60370.0,20933293.0,1525407.0,NaN,11890342.0,419402.0,18960.0,3931215.0,364527.0,NaN,2432946.0,35377.0
2022-12-01,59770.0,21987389.0,1322490.0,NaN,12101584.0,485313.0,20063.0,4444113.0,377113.0,NaN,2491803.0,27104.0
2023-01-01,63611.0,23132571.0,1369972.0,NaN,14379615.0,486309.0,20808.0,4040347.0,393904.0,NaN,2754632.0,149483.0
2023-02-01,58566.0,20435724.0,1098400.0,NaN,11799328.0,357345.0,19642.0,3942811.0,225253.0,NaN,2533307.0,24063.0
2023-03-01,62122.0,20237544.0,1191979.0,NaN,12261556.0,339707.0,20763.0,4015257.0,382678.0,NaN,2752975.0,36224.0


In [7]:
metrics.to_csv(FILENAME, sep="\t")